In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np

In [2]:
json_data = pd.read_json('data/world_bank_projects.json','r')
json_data.head()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,{'$oid': '52b213b38594d8a2be17c784'},2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,"FH,YW,YZ",IBRD,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


<b><font size = '3'>Json Exercise 1</font></b><br>
<i>1. Find the 10 countries with the most projects</i>


In [3]:
data = json_data.groupby(['countryshortname','countrycode']).size().sort_values(ascending  = False).head(10)
data

countryshortname    countrycode
Indonesia           ID             19
China               CN             19
Vietnam             VN             17
India               IN             16
Yemen, Republic of  RY             13
Nepal               NP             12
Bangladesh          BD             12
Morocco             MA             12
Mozambique          MZ             11
Africa              3A             11
dtype: int64

<b><font size = '3'>Json Exercise 2</font></b><br>
<i>2. Find the top 10 major project themes (using column 'mjtheme_namecode')</i>

In [4]:
with open('data/world_bank_projects.json', 'r') as json_file:
    json_data = json.load(json_file)
json_normalize(json_data, 'mjtheme_namecode').groupby(['code','name']).size().sort_values(ascending = False).head(10)

code  name                                        
11    Environment and natural resources management    223
10    Rural development                               202
8     Human development                               197
2     Public sector governance                        184
6     Social protection and risk management           158
4     Financial and private sector development        130
7     Social dev/gender/inclusion                     119
5     Trade and integration                            72
9     Urban development                                47
1     Economic management                              33
dtype: int64

<b><font size = '3'>Json Exercise 3</font></b><br>
<i>In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.</i>

In [5]:
'''create a function that builds a dictionary of theme codes and the corresponding theme names
use a regex pattern that searches for a non-blank name and adds the code as the key and the 
project name as the value in a blank dictionary
'''

import re
c_dict = {}
ptn = re.compile('\w')
def create_dict (row):
    global c_dict
    code = row['code']
    project = row['name']
    if ptn.match(project):
        c_dict[code] = project
    

In [6]:
'''Create a function that will replace the blank string in the name column with the correct theme name based on the 
theme code, using the c_dict created by the create_dict function'''

sp = re.compile('^$')
def replace_name (row):
    global c_dict
    code = row['code']
    project = row['name']
    if sp.match(project):
        project = c_dict[code]
    return project

In [7]:
'''import the data and check how many of the entries have blank/empty string theme names'''

with open('data/world_bank_projects.json', 'r') as json_file:
    json_data = json.load(json_file)
df = json_normalize(json_data, 'mjtheme_namecode')
df.name.value_counts()
len(df[df.name == ''])  #checking how many of the entries have blank theme names



122

In [8]:
'''apply the create_dic function to create the dictionary of them code : theme project name pairs
then apply the replace_name function to build iterate through the name column and replace the missing values
using .info() on df shows that there are no empty spots on the name column'''
df.apply(create_dict, axis = 1)
df.name = df.apply(replace_name, axis = 1)
len(df[df.name == ''])  #checking how many of the values in the name column has an empty string


0

In [9]:
df.head(10)

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
7,6,Social protection and risk management
8,7,Social dev/gender/inclusion
9,7,Social dev/gender/inclusion


In [10]:
import matplotlib.pyplot as plt
#df.set_index('code').groupby('name').size().sort_values().plot(kind = 'bar').legend()
#plt.show()
df.set_index('code').groupby('name').size().sort_values().plot(y = '# of Project' ,kind = 'barh', title = 'Top 5 Theme Counts')

